In [2]:
import os
import sys
from dotenv import load_dotenv

notebook_path = os.getcwd()

# One level up from the notebook
PROJECT_ROOT = os.path.abspath(os.path.join(notebook_path, "../"))

DATA_ROOT = os.path.join(PROJECT_ROOT, "data")

sys.path.insert(1, PROJECT_ROOT)
load_dotenv()

True

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(os.path.join(DATA_ROOT, "AAPL.O.csv"), index_col=0)
df.index = pd.to_datetime(df.index)

In [5]:
import yfinance as yf
import pandas as pd
from statsmodels.formula import api as sm
import plotly.graph_objects as go

In [6]:
# Define the tickers for the S&P 500 and the Magnificent 7
sp500_ticker = "^GSPC"
magnificent_7_tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "TSLA", "META"]

# Download the historical data for the S&P 500 and the Magnificent 7
data = yf.download(sp500_ticker + " " + " ".join(magnificent_7_tickers), period="6y")

# Extract the adjusted close prices
adj_close = data['Adj Close']

# Calculate daily returns
daily_returns = adj_close.pct_change().dropna()

# Rename columns for clarity
daily_returns.columns = [col.replace(' ', '_') for col in daily_returns.columns]
daily_returns.rename(columns={'^GSPC': 'SP500'}, inplace=True)

# Run regressions for each Magnificent 7 stock
betas = {}
for stock in magnificent_7_tickers:
    model = sm.ols(formula=f"{stock} ~ SP500", data=daily_returns).fit()
    betas[stock] = model.params['SP500']

# Print the betas
print("Betas of Magnificent 7 stocks with respect to S&P 500:")
for stock, beta in betas.items():
    print(f"{stock}: {beta:.3f}")

[*********************100%%**********************]  8 of 8 completed

Betas of Magnificent 7 stocks with respect to S&P 500:
AAPL: 1.182
MSFT: 1.180
GOOG: 1.135
AMZN: 1.092
NVDA: 1.800
TSLA: 1.574
META: 1.301


In [7]:
daily_returns

,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA,SP500
Date,,,,,,,,
2019-01-31,0.007201,0.028915,0.025077,0.108164,-0.018331,0.046291,-0.005668,0.008597
2019-02-01,0.000480,-0.053819,-0.005034,-0.005879,-0.015800,0.006817,0.016904,0.000899
2019-02-04,0.028405,0.004354,0.019851,0.021363,0.028800,0.030747,0.002178,0.006776
2019-02-05,0.017110,0.015612,0.011644,0.011285,0.013997,0.005162,0.027038,0.004708
2019-02-06,0.000344,-0.011183,-0.026841,-0.003914,-0.011099,0.020340,-0.012852,-0.002224
...,...,...,...,...,...,...,...,...
2025-01-23,-0.000759,0.001745,-0.002250,0.020770,0.001143,0.001020,-0.006577,0.005313
2025-01-24,-0.003935,-0.002421,0.011624,0.017346,-0.005932,-0.031246,-0.014065,-0.002855
2025-01-27,0.031780,0.002427,-0.040267,0.019135,-0.021394,-0.169682,-0.023193,-0.014581


In [8]:
# Calculate daily returns
daily_returns = adj_close.pct_change().dropna()

# Rename columns for clarity
daily_returns.columns = [col.replace(' ', '_') for col in daily_returns.columns]
daily_returns.rename(columns={'^GSPC': 'SP500'}, inplace=True)

# Calculate rolling betas with a 60-day window
window_size = 60
# Initialize an empty dictionary to store rolling betas
rolling_betas = {}
for stock in magnificent_7_tickers:
    rolling_betas[stock] = []

# Loop through the rolling windows
for i in range(window_size, len(daily_returns)):
    window = daily_returns.iloc[i - window_size:i]
    for stock in magnificent_7_tickers:
        model = sm.ols(formula=f"{stock} ~ SP500", data=window).fit()
        rolling_betas[stock].append(model.params['SP500'])

# Convert the rolling betas to a DataFrame with appropriate index
rolling_betas_df = pd.DataFrame(rolling_betas, index=daily_returns.index[window_size:])

# Plotting the rolling betas using Plotly
fig = go.Figure()
for stock in magnificent_7_tickers:
    fig.add_trace(go.Scatter(x=rolling_betas_df.index, y=rolling_betas_df[stock], mode='lines', name=stock))

fig.update_layout(title='Rolling Betas of Magnificent 7 Stocks',
                  xaxis_title='Date',
                  yaxis_title='Beta',
                  hovermode='x unified')

fig.show()